In [1]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

# Chat models

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [3]:
chat = ChatOpenAI(temperature=0.5)

In [4]:
system_message_prompt = "You are an expert annotator, you job is to annotate the given video pairs as True or False based on the description provided. You are also provided with the transcripts of the videos, use them while predicting the answer. \n\n"

In [5]:
human_message_prompt = HumanMessagePromptTemplate.from_template('''
Your job is to predict if the given  video A is a would follow prior to video B in a learning flow, give a very short discription in one line explaning your rational.
Give me the answer as 

False - The label should be "False" if:

Video A and Video B discuss related topics, but they do not naturally follow each other in an educational flow. These videos may have overlapping subject matter, but they are better suited for separate, non-sequential viewing.



True - The label should be "True" if:

Video A should precede Video B in a learning sequence to facilitate a more efficient and effective understanding of the topics discussed in both videos.
Video A and Video B, when viewed consecutively, form a cohesive narrative or educational sequence that enhances the understanding of a larger, interconnected subject, ensuring smooth and logical progression in the learning flow.

You are also provided with the optional transcripts of those videos, use them while predicting the answer. 


video A : {video_A}
Video A Transcript :   {video_A_transcript}

video B : {video_B}
Video B Transcript : {video_B_transcript}

Make use of the transcripts if provided, often times the video names maybe related but make sure to base your answer on the transcripts
'''
)

In [6]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# Import data

In [51]:
import pandas as pd
import numpy as np
import os
data = pd.read_csv(os.path.join('..', 'input', 'test.csv'))
data = data[['ID','pre requisite', 'concept', 'pre requisite taxonomy', 'concept taxonomy']]

# train_df = data[data['kfold'] != 1][['pre requisite', 'concept', 'label', 'pre requisite taxonomy', 'concept taxonomy']]
# dev_df = data[data['kfold'] == 1][['pre requisite', 'concept', 'label', 'pre requisite taxonomy', 'concept taxonomy']]
meta_data_transcript = pd.read_csv(os.path.join('..', 'input', 'metadata.csv'), index_col='video name')['transcript']
meta_data_extracted = pd.read_csv(os.path.join('..', 'input', 'metadata.csv'), index_col='video name')['extracted']

In [8]:
def get_gpt_response(video_A, video_A_transcript, video_B, video_B_transcript): 
    return chat(
        chat_prompt.format_prompt(
            video_A = video_A,
            video_A_transcript = video_A_transcript,
            video_B = video_B,
            video_B_transcript = video_B_transcript        
        ).to_messages()
    )

In [9]:
def apply_response(sample):
    video_A = sample['pre requisite']
    video_B = sample['concept']
    video_A_transcript = meta_data_transcript.get(video_A, 'Transcript not available')
    video_B_transcript = meta_data_transcript.get(video_B, 'Transcript not available')

    return get_gpt_response(
        video_A=video_A,
        video_B=video_B,
        video_A_transcript=video_A_transcript,
        video_B_transcript=video_B_transcript,
    ).content

numpy.int64

In [113]:
data = pd.read_csv(os.path.join('..', 'input', 'test.csv'))
data = data[['ID','pre requisite', 'concept', 'pre requisite taxonomy', 'concept taxonomy']]
data['response'] = ""
# save the data with response
data.to_csv(os.path.join('..', 'input', 'test_with_response.csv'), index=False)

In [104]:
data_response = pd.read_csv(os.path.join('..', 'input', 'test_with_response.csv'))
#chnage the datatype of the response column
data_response['response'] = data_response['response'].astype(str)

In [111]:
data_response.iloc[1]['response']

'Based on the given video transcripts, the answer is:\n\nTrue\n\nRationale: The video A is about Paper Chromatography, which explains how to separate different colored dyes present in black ink using paper chromatography. Video B is about Concentration, which explains the concept of concentration in solutions. Paper chromatography is a technique used to separate substances in a solution, which is a concept related to concentration. Therefore, it is logical to have a prior understanding of paper chromatography before learning about concentration in solutions. This sequence of learning would ensure a more efficient and effective understanding of the topics discussed in both videos.'

In [112]:
for _, sample in data_response.iterrows():
    id = int(sample['ID'])
    print(id)
    print(data_response.iloc[id-1]['response'] != 'nan')

1
True
2
True
3
True
4
True
5
True
6
True
7
True
8
True
9
True
10
True
11
True
12
True
13
True
14
True
15
True
16
True
17
True
18
True
19
True
20
True
21
True
22
True
23
True
24
True
25
True
26
True
27
True
28
True
29
True
30
True
31
True
32
True
33
True
34
True
35
True
36
True
37
True
38
True
39
True
40
True
41
True
42
True
43
True
44
True
45
True
46
True
47
True
48
True
49
True
50
True
51
True
52
True
53
True
54
True
55
True
56
True
57
True
58
True
59
True
60
True
61
True
62
True
63
True
64
True
65
True
66
True
67
True
68
True
69
True
70
True
71
True
72
True
73
True
74
True
75
True
76
True
77
True
78
True
79
True
80
True
81
True
82
True
83
True
84
True
85
True
86
True
87
True
88
True
89
True
90
True
91
True
92
True
93
True
94
True
95
True
96
True
97
True
98
True
99
True
100
True
101
True
102
True
103
True
104
True
105
True
106
False
107
False
108
False
109
False
110
False
111
False
112
False
113
False
114
False
115
False
116
False
117
False
118
False
119
False
120
False
121
False
122

ValueError: invalid literal for int() with base 10: 'quations'

In [107]:
data_response.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      615 non-null    object
 1   pre requisite           615 non-null    object
 2   concept                 615 non-null    object
 3   pre requisite taxonomy  614 non-null    object
 4   concept taxonomy        614 non-null    object
 5   response                615 non-null    object
dtypes: object(6)
memory usage: 29.0+ KB


In [77]:
#chnage the datatype of the response column
data_response['response'] = data_response['response'].astype(str)

In [41]:
data['response'] = data.apply(apply_response, axis=1)

KeyboardInterrupt: 

In [114]:
import pandas as pd
data_response = pd.read_csv(os.path.join('..', 'input', 'test_with_response.csv'))
#chnage the datatype of the response column
data_response['response'] = data_response['response'].astype(str)

In [115]:
def classify(response):
    if 'True' in response : return 1
    else: return 0


In [116]:
data_response['label'] = data_response['response'].apply(classify)

In [117]:
data_classified = data_response[['ID', 'label']]

In [118]:
data_classified.to_csv(os.path.join('..', 'output', 'test_classified-2.csv'), index=False)